In [1]:
import tensorflow as tf
from keras import layers
import keras

In [82]:
import tensorflow as tf
from tensorflow.keras import layers
inputs = tf.random.uniform((1,10,), maxval=1000, dtype=tf.dtypes.int32)
inputs
embed = layers.Embedding(1000, 4)(inputs)
embed
rnn_out = layers.LSTM(8)(embed)
rnn_out
rnn_out = layers.LSTM(8, return_sequences=True, return_state=False)(embed)
rnn_out
rnn_out, state_h, state_c = layers.LSTM(8, return_sequences=True, return_state=True)(embed)

# print(rnn_out)
# print(state_h)
# print(state_c)
rnn_out = layers.GRU(8, return_sequences=True, return_state=True)(embed)
# print(rnn_out)

out = layers.Bidirectional(layers.GRU(8, return_sequences=True, return_state=True))(embed)
out
out = layers.Bidirectional(layers.GRU(8, return_sequences=True, return_state=True))(out)
out

[<tf.Tensor: shape=(1, 10, 16), dtype=float32, numpy=
 array([[[-7.3322174e-03, -1.0034279e-02, -2.8253892e-03, -8.7888649e-04,
          -5.0847367e-03,  8.1342109e-04,  2.8604991e-03, -5.4657990e-03,
          -1.0536960e-02, -1.7370414e-03, -5.3907605e-03, -2.0430377e-03,
           7.6764044e-03, -5.6762912e-04,  6.5477211e-03,  4.0629935e-03],
         [-9.6790539e-03, -1.1318546e-02, -1.4235536e-03,  5.8886496e-04,
          -3.2291999e-03,  2.0670581e-03,  6.8934737e-03, -7.0039923e-03,
          -6.5036281e-03, -9.4519980e-04,  1.6114360e-04, -1.0412757e-03,
           1.0327345e-02, -5.3445058e-04,  1.6140345e-03,  8.4834313e-04],
         [-5.4624081e-03, -4.6583535e-03,  6.5508875e-04, -2.3658287e-03,
           4.3014940e-03,  1.2340078e-03,  4.5494474e-03, -1.0967149e-02,
          -1.1436666e-02, -1.3763028e-03, -3.8594988e-03,  3.3573342e-05,
           2.2041256e-02,  3.2485952e-03, -1.0667082e-02,  8.3156757e-04],
         [-5.5646151e-03, -7.2069844e-04,  4.2092144e-0

In [5]:
inputs = layers.Input((10, ), batch_size=1)
embed = layers.Embedding(1000, 100)(inputs)

In [6]:
out = layers.GRU(512, return_sequences=True, return_state=True)(embed)

In [11]:
bigru_out, forward_s, backward_s = layers.Bidirectional(layers.GRU(512, return_state=True))(embed)

In [20]:
encoder_state = layers.Concatenate([forward_s, backward_s])

In [21]:
dec_inputs = layers.Input((1, ), batch_size = 1)
dec_embed = layers.Embedding(1000, 128)(dec_inputs)

In [30]:
from tensorflow.keras import Input

inputs = Input((10,), batch_size=1)

In [31]:
embed = layers.Embedding(1000, 128)(inputs)
embed

<tf.Tensor 'embedding_10/embedding_lookup/Identity_1:0' shape=(1, 10, 128) dtype=float32>

In [34]:
bigru_out, forward_s, backward_s = layers.Bidirectional(layers.GRU(512, 
                                            return_sequences=True, 
                                            return_state=True))(embed)
bigru_out

<tf.Tensor 'bidirectional_13/concat:0' shape=(1, 10, 1024) dtype=float32>

In [37]:
encoder_state = layers.Concatenate()([forward_s, backward_s])
encoder_state

decoder_prev_state = encoder_state

In [ ]:
# st-1 -> encoder_state
# hj -> bigru_out

In [38]:
out1 = layers.Dense(256)(decoder_prev_state)
out2 = layers.Dense(256)(bigru_out)

In [40]:
bigru_out.shape

TensorShape([1, 10, 1024])

In [41]:
embed.shape

TensorShape([1, 10, 128])

In [42]:
out1 = tf.expand_dims(out1, axis=1)
out1

<tf.Tensor 'ExpandDims:0' shape=(1, 1, 256) dtype=float32>

In [56]:
out = out1 + out2
out

<tf.Tensor 'AddV2_2:0' shape=(1, 10, 256) dtype=float32>

In [57]:
out = layers.Activation("tanh")(out)

In [58]:
out = layers.Dense(1)(out)
out #논문에서 alpha 부분

<tf.Tensor 'dense_5/BiasAdd:0' shape=(1, 10, 1) dtype=float32>

In [53]:
alpha = tf.squeeze(out, axis=-1)
alpha

<tf.Tensor 'Squeeze_3:0' shape=(1, 10) dtype=float32>

In [60]:
alpha = layers.Activation("softmax")(out)
alpha

<tf.Tensor 'activation_5/truediv:0' shape=(1, 10, 1) dtype=float32>

In [55]:
alpha.shape, bigru_out.shape

(TensorShape([1, 10]), TensorShape([1, 10, 1024]))

In [71]:
context = tf.reduce_sum(alpha*bigru_out, axis=1, keepdims=True)
context

<tf.Tensor 'Sum_3:0' shape=(1, 1, 1024) dtype=float32>

In [69]:
dec_embed.

TensorShape([1, 1, 128])

In [73]:
dec_input = tf.concat([context, dec_embed], axis=-1)
dec_input

<tf.Tensor 'concat:0' shape=(1, 1, 1152) dtype=float32>

In [79]:
dec_out, dec_state = layers.GRU(512, return_sequences=True, return_state=True)(dec_input)
dec_out

<tf.Tensor 'gru_21/PartitionedCall:1' shape=(1, 1, 512) dtype=float32>

In [80]:
prediction = layers.Dense(1000, activation='softmax')(dec_out)
prediction

<tf.Tensor 'dense_7/truediv:0' shape=(1, 1, 1000) dtype=float32>